In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/earthquakedatasetforindiansubcontinent/clead_earth_quake_data.csv',names=["index","datetime","nearest_volcano_distance","location","latitude","longitude","magnitude","depth"],header=None)

In [ ]:
df=df.drop(0,axis=0)
df = df.set_index('index')
df.index = df.index.astype(int)

In [ ]:
def DateTimePreprocessing(s):
    
    matched = re.search(r"\(\d\d.*?\)",s)
    if matched is not None:
        date = re.search(r'\((.*?)\)',matched.group(0)).group(1)
        return datetime.strptime(date.replace(' ','-'),"%d-%b-%Y-%H:%M:%S-GMT")
    else:
        date = re.search(r'\((\d.*?)\)',s)
        if date is not None:
            
            return datetime.strptime(date.group(0).replace(' ','-'),"(%d-%b-%Y-%H:%M:%S-GMT)")
        else:
            return datetime.strptime(s.replace(' ','-'),"%d-%b-%Y-%H:%M:%S-GMT")

In [ ]:
df.datetime=df.datetime.apply(lambda s: pd.to_datetime(DateTimePreprocessing(s)))

In [ ]:
df.depth=df.depth.apply(lambda s: re.search("\d+\.\d+", s).group(0)).astype('float32')

In [ ]:
df.latitude = df.latitude.astype('float32')
df.longitude = df.longitude.astype('float32')
df.magnitude = df.magnitude.astype('float32')



df.nearest_volcano_distance=df.nearest_volcano_distance.apply(lambda s: re.search(r'\((.*?)\)',s).group(1))

In [ ]:
df.nearest_volcano_distance=df.nearest_volcano_distance.apply(lambda s: re.search("\d+", s).group(0)).astype('int')

In [ ]:
df.nearest_volcano_distance

In [ ]:
df.dtypes

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(data=df,y="datetime",x="magnitude")

In [ ]:
df.plot(x="datetime",y="depth")

In [ ]:
sns.heatmap(df.corr())

In [ ]:
!pip install fbprophet

In [ ]:
p = df[df.location.duplicated()]
p.plot(x="datetime",y="magnitude")

In [ ]:
sns.countplot(data=p,x="magnitude")

In [ ]:
data=df
data.datetime = data.datetime.astype('string')

# Show the earthquake time frame animation with Geoplot?

In [ ]:
import plotly.express as px

px.scatter_geo(data,lat='latitude',lon='longitude',
               scope="asia",animation_frame="datetime",
               hover_name="location",color='depth',size='magnitude')


In [ ]:
from fbprophet import Prophet

In [ ]:
data = df[['datetime','magnitude']]
data.columns = ['ds','y']

In [ ]:
m = Prophet()
m.fit(data)
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


# How powerful earthquake can happen next 1 year?

In [ ]:

solution = m.plot(forecast)


# Find the seasonality and trend of the earthquake

In [ ]:
solution1 = m.plot_components(forecast)
